<a href="https://colab.research.google.com/github/Ekaterina-31/MIFI-Data-analysis/blob/main/PYTHON_%D0%B4%D0%BB%D1%8F_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_HW_%E2%84%96_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
git remote add origin https://github.com/Ekaterina-31/MIFI-HW.git
git branch -M main
git push -u origin main

SyntaxError: invalid syntax (<ipython-input-1-3ffdd6e9b9b0>, line 1)

In [ ]:
!pip install seaborn
!pip install matplotlib

In [ ]:
# Загружаем файлы через Google Drive, чтобы избежать временных ограничений на загрузку файлов
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Читаем csv-файлы из Google Drive. Из файла train.csv читаем только первые 1000000 строк
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python для анализа данных/HW № 1/train.csv', nrows=1000000)
lectures_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python для анализа данных/HW № 1/lectures.csv')
questions_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python для анализа данных/HW № 1/questions.csv')

MessageError: Error: credential propagation was unsuccessful

Исследуем структуру данных в файле train.csv с информацией о прохождении тестов студентами на образовательной платформе:

In [ ]:
# Выводим первые 5 строк таблицы
display(train_df.head())

In [ ]:
# Определяем количество строк и столбцов в таблице
train_df.shape

In [ ]:
# Выводим более детальную информацию о столбцах таблицы (тип данных в каждом столбце таблицы, количество непустых значений в каждом столбце таблицы)
train_df.info()

In [ ]:
# Выводим количество пустых значений в каждом столбце таблицы
train_df_NaN = train_df.isnull().sum().reset_index()
train_df_NaN.columns = ['Показатель', 'Количество_NaN']
display(train_df_NaN)

Из детальной информации о столбцах таблицы следует, что непустые значения содержатся в колонке prior_question_elapsed_time, где отражается среднее время, которое потребовалось пользователю, чтобы ответить на каждый вопрос в предыдущем наборе вопросов, и в колонке prior_question_had_explanation с данными смотрел или нет пользователь объяснение и правильный ответ после ответа на предыдущий пакет вопросов

In [ ]:
# Выводим на экран значение статистических параметров для столбцов таблицы из файла train.csv
train_df.describe().loc[:, ['content_type_id', 'answered_correctly','prior_question_elapsed_time']]

In [ ]:
# Выводим на экран значение статистических параметров для столбца с типом данных 'object'
train_df.describe(include=['object'])

Статистические параметры столбцов таблицы из файла train.csv показывают:


*   взаимодействие с пользователем в основном осуществляется через вопросы, заданные пользователю (значение показателя content_type_id = 0), нежели через лекции, которые смотрит пользователь (значение показателя content_type_id = 1). Об этом свидетельствуют среднее значение и квантили показателя content_type_id, которые близки к 0
*   большая часть ответов пользователей на вопросы правильные. При этом для точного расчета доли правильных ответов требуется исключить из поля answered_correctly значения -1 для лекций. Расчет доли правильных ответов приведен ниже  
*   в среднем время, затраченное на ответ на вопрос, составило 25 секунд. При этом стандартное отклонение составило 20 сек
*   большинство пользователей смотрели объяснение и правильный ответ после ответа на предыдущий пакет вопросов, так как наболее частое значение показателя prior_question_had_explanation = True







Аналогично исследуем структуру данных в файлах с лекциями и вопросами:

In [ ]:
# Выводим первые 5 строк таблиц
display(lectures_df.head())
display(questions_df.head())

In [ ]:
# Определяем количество строк и столбцов в таблицах
display(lectures_df.shape, questions_df.shape)

In [ ]:
# Выводим более детальную информацию о столбцах таблиц
lectures_df.info()
questions_df.info()

In [ ]:
# Выводим на экран значение статистических параметров для столбцов таблиц
display(lectures_df.describe(), questions_df.describe())

In [ ]:
# Выводим на экран значение статистических параметров для столбцов с типом данных 'object'
display(lectures_df.describe(include=['object']), questions_df.describe(include=['object']))

In [ ]:
# Выведем темы лекций
lectures_df['type_of'].unique()

Проанализируем характеристики, влияющие на успеваемость:

1. Процент правильно отвеченных вопросов

In [ ]:
# Выведем значения, которые содержатся в столбце answered_correctly
train_df['answered_correctly'].value_counts()

In [ ]:
# Значение показателя answered_correctly -1 относится к лекциям
# Ислючаем строки для лекций из таблицы для корректного расчета процента правильно отвеченных вопросов
train_df[train_df['content_type_id'] == 0]['answered_correctly'].value_counts(normalize=True)

Вывод: пользователи ответили правильно на 65% вопросов

2. Влияние времени на правильность ответа

In [ ]:
# Построим гистограмму, где по оси x отразим время, затраченное на ответ, по оси y количество ответов на вопросы
sns.histplot(train_df['prior_question_elapsed_time'], kde = True)
plt.title('Распределение времени на ответы')
plt.show()

Из постороенной гистограммы видно, что в среднем время ответа составляет 20-30 секунд

In [ ]:
# Рассчитаем среднее арифметическое значение времени на ответ
train_df['prior_question_elapsed_time'].mean()

In [ ]:
# Выведем отдельно среднее арифметическое значение времени, затраченного на верный и на неверный ответ
train_df.groupby('answered_correctly')['prior_question_elapsed_time'].mean()

Вывод: среднее время, затраченное на верный и на неверный ответ, отличается незначительно. Из этого можно сделать выврд, что время, затаченное на ответ не оказало существенного влияния на успеваемость студентов

3. Влияние объяснений на правильность ответа

In [ ]:
# Выведем средее количество правильных ответов для случаев, когда студены смотрели объяснения и правильные ответы и когда не смотрели
# Строки с данными по лекциям (content_type_id = 0) не учитываем
explanation_correct = train_df[train_df['content_type_id'] == 0].groupby('prior_question_had_explanation')['answered_correctly'].mean()
display(explanation_correct)

Вывод: разъяснения положительно влияют на успеваемость студентов. Если учащийся получал разъяснения, то отвечал правильно на вопросы в 67% случаев, иначе только в 49%

4. Зависмость успеваемости от темактики, сложности вопросов

In [ ]:
# Объединим данные из файлов train.csv и questions.csv
train_questions_df = pd.merge(train_df[train_df['content_type_id'] == 0], questions_df, left_on = 'content_id', right_on = 'question_id')
train_questions_df.head()

In [ ]:
# Анализ по тегам
# Выведем среднее количество верных ответов по тегу или набору тегов
tag_q_correctness = train_questions_df.groupby('tags')['answered_correctly'].mean()
display(tag_q_correctness)

In [ ]:
# Анализ по разделам теста (part)
# Выведем среднее количество верных ответов в зависимости от раздела теста
part_q_correctness = train_questions_df.groupby('part')['answered_correctly'].mean()
display(part_q_correctness)

Вывод: анализ по тегам и разделам теста показывает, что по различным темам, разделам теста среднее значение верных ответ отличается. На это могут влиять сложность темы и другие факторы, что также отражается на успеваемости студентов

5. Роль лекций и объяснений в обучении

In [ ]:
# Создадим копию таблицы из файла train.csv, где заменим -1, NaN и False на 0
train_df_copy = train_df.replace({-1: 0, 'NaN': 0, False: 0})
# Выведем информацию о количестве лекций, которые посетил студент, количестве верных ответов и количестве полученных разъяснений
# Эту информацию выводим в разрезе пользователей
train_lectures_explanations = train_df_copy.groupby('user_id')[['content_type_id', 'answered_correctly', 'prior_question_had_explanation']].sum().reset_index().rename(columns={'content_type_id': 'lectures'})
display(train_lectures_explanations)

In [ ]:
# В продолжение анализа, приведенного в пункте 3, построим график, чтобы выявить зависимость между полученными разъяснениями и количеством верных ответов
train_lectures_explanations.plot(x='prior_question_had_explanation', y='answered_correctly', kind = 'scatter')

Вывод: на графике видим четкую зависимость между количеством полученных разъяснений и количеством верных ответов, что показывает существенное влияние этого фактора на успеваемость студентов

In [ ]:
# Построим график, чтобы выявить зависимость между колиеством лекций, которые посетил студент, и количеством верных ответов
train_lectures_explanations.plot(x='lectures', y='answered_correctly', kind = 'scatter')

Вывод: на представленном графике мы также видим, что прослушивание лекций также положительно влияет на успеваемость, хотя зависимость не такая четкая как в случае с разъяснениями. Это может говорить о высом уровне подготовки студентов, который позволяем им успешно проходить тест без лекций, возможно студенты пользуются другими источниками информации кроме лекций на платформе и тп